In [5]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

### 1. Data

In [6]:
#Dane:

data = {
    'size': ['XL', 'L', 'M', 'L', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']}

df_raw = pd.DataFrame(data)

# kopia zapasowa
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


### 2. Klasa OneHotEncoder  - kodowanie zmiennej kategorialnej df['size']

In [29]:
# tworzenie instancji klasy:

"""
Parametr 'sparse=False':

Domyślnie klasa OneHotEncoder zwraca macierz rzadką - nie jesteśmy w stanie podejrzeć jak ona wygląda. 
Z parametrem 'true' uzyskujemy  tylko informację, że jest to macierz rzadka. 

<5x3 sparse matrix of type '<class 'numpy.float64'>'
    with 5 stored elements in Compressed Sparse Row format>

Jeśli mamy macierz i bardzo dużo zmiennych 
i prawie same 0,1 - nie ma sensu przetrzymywać tych wszytskich danych w pamięci - wystarczy 1 (logiczne, ze reszta to 0)


"""

encoder = OneHotEncoder(sparse=False)

# metoda fit

encoder.fit(df[['size']])                # tutaj dwa nawiasy kwadratowe żeby było dataframe

OneHotEncoder(sparse=False)

In [31]:
# macierz numpy array

"""
Jak czytać powstałą tablicę:

- tablica ma 3 kolumny i 5 wierszy
- po wywołaniu atrybutu 'categories_' = array(['L', 'M', 'XL'] - to są kolumny
- tam gdzie jest np. L dostajemy 1, a w innym przypadku 0
- XL wystąpiło tylko raz - w pierwszym wierszu - więc 1, a dalej same 0

"""

encoder.transform(df[['size']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [21]:
# atrybut categories_

encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

#### - argument 'drop=first'

In [37]:
"""
Przy takim kodowaniu możemy popełnić błąd - w powyższym przykładzie powstaje zależność liniowa (w wierszach ?). 
Gdy w pierwszym jest 1, w pozostałych 0 itd. - powstaje zależność liniowa.
Logiczne jest, jeśli w dwóch powstałych kolumna są dwie wartości 0, to w trzeciej musi być 1


Mapowanie cech porzadkowych

Czy w tym przypadku size nie powinno być zmapowane: {'M':1,'L':2,'XL':3} jako cecha porządkowa? 


Dzień dobry,

Istnieje wiele sposobów kodowania zmiennych kategorialnych jako liczb i wykorzystywania ich w algorytmach uczenia maszynowego.

Kodowanie porządkowe wykonuje się, aby zapewnić, że kodowanie zmiennej zachowuje porządkowy charakter zmiennej. 
Często nie wpływa to znacząco na model, zwłaszcza w uczeniu głębokim.

Chociaż kodowanie jest proste, to przy większej ilości zmiennych kategorycznych (np. ponad 100) aby podać wartości porządkowe 
i jakie jest rzeczywiste odwzorowanie z tekstu na liczbę całkowitą zgodnie z kolejnością zajmie sporo czasu. Często ten czas 
nie jest odzwierciedlony w realnej poprawie działania modelu.

Podsumowując, można stosować zarówno jedną metodę i drugą. Pierwsza jest zdecydowanie szybsza. Warto zawsze spojrzeć 
indywidualnie na przypadek i porównać rezultaty.

"""


"\nPrzy takim kodowaniu możemy popełnić błąd - w powyższym przykładzie powstaje zależność liniowa (w wierszach ?). \nGdy w pierwszym jest 1, w pozostałych 0 itd. - powstaje zależność liniowa.\nLogiczne jest, jeśli w dwóch powstałych kolumna są dwie wartości 0, to w trzeciej musi być 1\n\n\n\n\n\nMapowanie cech porzadkowych\n\nCzy w tym przypadku size nie powinno być zmapowane: {'M':1,'L':2,'XL':3} jako cecha porządkowa? \n\n\nDzień dobry,\n\nIstnieje wiele sposobów kodowania zmiennych kategorialnych jako liczb i wykorzystywania ich w algorytmach uczenia maszynowego.\n\nKodowanie porządkowe wykonuje się, aby zapewnić, że kodowanie zmiennej zachowuje porządkowy charakter zmiennej. \nCzęsto nie wpływa to znacząco na model, zwłaszcza w uczeniu głębokim.\n\nChociaż kodowanie jest proste, to przy większej ilości zmiennych kategorycznych (np. ponad 100) aby podać wartości porządkowe \ni jakie jest rzeczywiste odwzorowanie z tekstu na liczbę całkowitą zgodnie z kolejnością zajmie sporo czasu. 

In [38]:
# Jeżeli mamy n różnych wartości zmiennej kategorycznej, to po kodowaniu 0,1, mamy n-1 kolumn, które zostały zakodowane


encoder = OneHotEncoder(drop='first', sparse=False)
encoder.fit(df[['size']]) 
encoder.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [36]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

### 3. Pandas - get_dummies() - dobra funkcja i chyba lepsza od sklearn według autora ( w niektórych przypadkach lepsza sklearn):

##### - funkcja ta koduje wszystkie zmienne kategoryczne na 0,1

In [39]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [43]:
"""
Zmienna 'bought'(kodowana w poprzedniej lekcji za pomocą klasy 'LabelEncoder') została zmapowana na dwie kolumny, a wystarczy 
jedna - co jest trochę bez sensu. Gdy mamy dwie unikalne wartości - kolumna będzie jedna (po kodowaniu n = n-1 kolumn)



"""






pd.get_dummies(data = df)

,price,weight,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male,bought_no,bought_yes
0,199.0,500,0,0,1,0,0,1,1,0,0,1
1,89.0,450,1,0,0,0,1,0,0,1,1,0
2,99.0,300,0,1,0,1,0,0,0,1,0,1
3,129.0,380,1,0,0,0,1,0,1,0,1,0
4,79.0,410,0,1,0,0,0,1,1,0,0,1


In [45]:
# drop='first', jak w przypadku klasy OneHotEncoder, usuwą jedną niepotrzebną kolumnę:

pd.get_dummies(data = df, drop_first=True)

,price,weight,size_M,size_XL,color_green,color_red,gender_male,bought_yes
0,199.0,500,0,1,0,1,0,1
1,89.0,450,0,0,1,0,1,0
2,99.0,300,1,0,0,0,1,1
3,129.0,380,0,0,1,0,0,0
4,79.0,410,1,0,0,1,0,1


In [47]:
# atrybut prefix

pd.get_dummies(data = df, drop_first=True, prefix='new')

,price,weight,new_M,new_XL,new_green,new_red,new_male,new_yes
0,199.0,500,0,1,0,1,0,1
1,89.0,450,0,0,1,0,1,0
2,99.0,300,1,0,0,0,1,1
3,129.0,380,0,0,1,0,0,0
4,79.0,410,1,0,0,1,0,1


In [48]:
# kodowanie tylko wybranych kolumn

pd.get_dummies(data = df, drop_first=True, columns=['size'])

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500,yes,0,1
1,green,male,89.0,450,no,0,0
2,blue,male,99.0,300,yes,1,0
3,green,female,129.0,380,no,0,0
4,red,female,79.0,410,yes,1,0
